#### Import necessary packages

In [3]:
import pandas as pd
import numpy as np
import math
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from sklearn.metrics import mean_squared_error as mse
from sklearn.metrics import mean_absolute_error as mae

#### Load the datasets

In [4]:
X_train=pd.read_csv("X_train.csv")
y_train=pd.read_csv("y_train.csv")

In [5]:
X_val=pd.read_csv("X_val.csv")
y_val=pd.read_csv("y_val.csv")

In [98]:
X_test=pd.read_csv("X_test.csv")
y_test=pd.read_csv("y_test.csv")

In [103]:
sample_submission1=pd.read_csv("recruit-restaurant-visitor-forecasting/sample_submission.csv/sample_Submission.csv")

In [6]:
X_train=X_train.values.reshape(X_train.shape[0],1,13)
X_val=X_val.values.reshape(X_val.shape[0],1,13)

In [65]:
X_val=X_val.values.reshape(X_val.shape[0],1,13)

In [96]:
# create and fit the LSTM network
def create_LSTM(X,y):
    model = Sequential()
    model.add(LSTM(100, input_shape=(1, X.shape[2])))
    model.add(Dense(10))
    model.add(Dense(1))
    model.compile(loss='mean_squared_error', optimizer='adam')
    model.fit(X, np.log1p(y['visitors']), epochs=100, batch_size=1000, verbose=2)

In [97]:
create_LSTM(X_train,y_train)

Epoch 1/100
236/236 - 3s - loss: 0.8993
Epoch 2/100
236/236 - 3s - loss: 0.6498
Epoch 3/100
236/236 - 3s - loss: 0.6499
Epoch 4/100
236/236 - 3s - loss: 0.6500
Epoch 5/100
236/236 - 3s - loss: 0.6500
Epoch 6/100
236/236 - 3s - loss: 0.6500
Epoch 7/100
236/236 - 3s - loss: 0.6500
Epoch 8/100
236/236 - 3s - loss: 0.6500
Epoch 9/100
236/236 - 3s - loss: 0.6501
Epoch 10/100
236/236 - 4s - loss: 0.6502
Epoch 11/100
236/236 - 4s - loss: 0.6501
Epoch 12/100
236/236 - 3s - loss: 0.6505
Epoch 13/100
236/236 - 4s - loss: 0.6507
Epoch 14/100
236/236 - 3s - loss: 0.6507
Epoch 15/100
236/236 - 3s - loss: 0.6504
Epoch 16/100
236/236 - 4s - loss: 0.6503
Epoch 17/100
236/236 - 4s - loss: 0.6505
Epoch 18/100
236/236 - 3s - loss: 0.6505
Epoch 19/100
236/236 - 4s - loss: 0.6505
Epoch 20/100
236/236 - 3s - loss: 0.6506
Epoch 21/100
236/236 - 3s - loss: 0.6512
Epoch 22/100
236/236 - 3s - loss: 0.6506
Epoch 23/100
236/236 - 4s - loss: 0.6502
Epoch 24/100
236/236 - 5s - loss: 0.6504
Epoch 25/100
236/236 - 4s

In [85]:
y_pred = model.predict(X_train)

In [86]:
y_pred_val = model.predict(X_val)

In [84]:
def rmsle(y_true,y_pred): 
    """
    Compute the Root Mean Squared Log Error for prediction y_pred and targets y_true

    Args:
        y_pred - numpy array containing predictions 
        y_true - numpy array containing targets 
    """
    return np.sqrt(np.square(np.log(y_pred + 1) - np.log(y_true + 1)).mean())

#### Evaluation metrics for both train and validation data

In [95]:
from prettytable import PrettyTable
    
x = PrettyTable()

x.field_names = ["RandomForestRegressor", "RMSLE", "RMSE","MSE","MAE"]

x.add_row(["Train data",rmsle(y_true=(y_train['visitors'].values.reshape(-1,1)),y_pred=np.expm1(y_pred)),
           math.sqrt(mse(np.expm1(y_pred),(y_train['visitors']))),mse(np.expm1(y_pred),
            (y_train['visitors'])),mae(np.expm1(y_pred),(y_train['visitors']))])
x.add_row(["Validation data",rmsle(y_true=(y_val['visitors'].values.reshape(-1,1)),y_pred=np.expm1(y_pred_val)),
           math.sqrt(mse(np.expm1(y_pred_val),(y_val['visitors']))),mse(np.expm1(y_pred_val),
            (y_val['visitors'])),mae(np.expm1(y_pred_val),(y_val['visitors']))])

print(x)
    

+-----------------------+--------------------+--------------------+--------------------+--------------------+
| RandomForestRegressor |       RMSLE        |        RMSE        |        MSE         |        MAE         |
+-----------------------+--------------------+--------------------+--------------------+--------------------+
|       Train data      | 0.8060785356800454 | 17.511436978421315 | 306.65042504922144 | 12.105487897949622 |
|    Validation data    | 0.8307919865463302 | 19.325348481397057 | 373.46909392743555 | 12.825793219061973 |
+-----------------------+--------------------+--------------------+--------------------+--------------------+


#### Predict for sample submission data and store it in .csv file for kaggle submission

In [100]:
X_test=X_test.values.reshape(X_test.shape[0],1,13)
y_test_pred=model.predict(X_test)
sample_submission1['visitors']=(np.expm1(y_test_pred)).astype(int)
sample_submission1.to_csv("Sub_lstm.csv",index=False)